# **CLIP BIAS ANALYSIS E DEBIASING**

# *Setup Colab, Models and Daset*

In [1]:
# -----------------------------------------------
# STEP 1: SETUP AMBIENTE E CONFIGURAZIONE
# (Codice fornito dall'utente, nessuna modifica)
# -----------------------------------------------

# 1.1 Verifica della GPU
print("--- 1.1 Verifica GPU ---")
!nvidia-smi
print("-" * 30)

# 1.2 Installazione delle librerie necessarie
print("--- 1.2 Installazione librerie ---")
!pip install transformers peft datasets scikit-learn -q
print("Librerie installate con successo.")
print("-" * 30)

# 1.3 Montaggio di Google Drive
print("--- 1.3 Montaggio Google Drive ---")
from google.colab import drive
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive montato con successo.")
except Exception as e:
    print(f"Errore nel montaggio di Google Drive: {e}")
print("-" * 30)

# -----------------------------------------------
# STEP 1.4: Import, Costanti e Gestione Dati Locali
# (MODIFICATO per gestire la copia locale dei dati)
# -----------------------------------------------
print("--- 1.4 Import e Costanti ---")
import torch
import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm.auto import tqdm # Utile per i loop di valutazione
import random
import json
import seaborn as sns
import torchvision
# Assicurati che l'import sia corretto (con 3 'T')
try:
    from torchvision.datasets.oxford_iiit_pet import OxfordIIITPet
except ImportError:
    print("ATTENZIONE: Import 'OxfordIIITPet' non riuscito. Riprovo con 'OxfordIIITPet'...")
    try:
        from torchvision.datasets import OxfordIIITPet
    except ImportError as e:
        print(f"Errore import dataset Pet: {e}. Assicurati che torchvision sia installato e aggiornato.")

from torch.utils.data import Dataset, DataLoader
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from torch.nn.functional import cosine_similarity, relu

# Import per i prossimi step
from transformers import CLIPModel, CLIPProcessor
from peft import LoraConfig, get_peft_model, PeftModel

# --- 1.4.1 Configurazione del Device ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device in uso: {DEVICE}")

# --- 1.4.2 Costanti del Modello ---
MODEL_ID = "openai-community/clip-rn50"

# --- 1.4.3 Path di Base (Google Drive) ---
# Questo path punta alla radice del tuo progetto su Drive
BASE_DRIVE_PATH = Path("/content/drive/MyDrive/clip_debiasing/")

# --- 1.4.4 Path Locali (Disco Veloce Colab) ---
# Definiamo dove salvare i dati sul disco locale di Colab
LOCAL_DATA_DIR = Path("/content/datasets")
LOCAL_DATA_DIR.mkdir(exist_ok=True)
print(f"I dataset locali saranno usati da: {LOCAL_DATA_DIR}")

# Path al dataset Pexels (Locale)
PEXELS_DIR = LOCAL_DATA_DIR / "pexels_evaluation_dataset"
PEXELS_CSV = PEXELS_DIR / "annotations.csv"
PEXELS_IMG_DIR = PEXELS_DIR / "images"

# Path al dataset OxfordIIIPet (Locale)
PETS_DIR = LOCAL_DATA_DIR / "Oxford_IIIT_Pet"
PETS_IMG_DIR = PETS_DIR / "images"
PETS_ANNO_DIR = PETS_DIR / "annotations"

# Definiamo i path per FairFace (Locale)
FAIRFACE_DIR = LOCAL_DATA_DIR / "fairface_eval"
FAIRFACE_VAL_CSV = FAIRFACE_DIR / "fairface_label_val.csv"
FAIRFACE_IMG_DIR = FAIRFACE_DIR

# Definiamo i path per il Challenge Set (Locale)
CHALLENGE_DIR = LOCAL_DATA_DIR / "challenge_dataset" # La radice è /content/datasets
CHALLENGE_CSV = CHALLENGE_DIR / "annotations_challenge.csv"
CHALLENGE_IMG_DIR = CHALLENGE_DIR / "challenge_images_val"

# --- 1.4.5 Path di Output (su Drive) ---
OUTPUT_DIR = BASE_DRIVE_PATH / "outputs_rn50" # <-- MODIFICATO
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"I risultati e i grafici verranno salvati in: {OUTPUT_DIR}")

# -----------------------------------------------
# 1.5 Decompressione Dataset (FORZATA E CORRETTA)
# -----------------------------------------------
print("\n--- 1.5 Pulizia Forzata e Decompressione Dataset Locali ---")

# 1. PULIZIA FORZATA: Rimuoviamo tutto per sicurezza
# Questo risolve qualsiasi stato "corrotto" o parziale.
print(f"Pulizia forzata di: {LOCAL_DATA_DIR}")
!rm -rf "{LOCAL_DATA_DIR}"

# 2. Ricreiamo la cartella base
LOCAL_DATA_DIR.mkdir(exist_ok=True)

# Definiamo i path dei file .ZIP sul tuo Drive
PEXELS_ZIP_PATH = BASE_DRIVE_PATH / "data/pexels_evaluation_dataset.zip"
PETS_ZIP_PATH = BASE_DRIVE_PATH / "data/oxford-iiit-pet.zip"
FAIRFACE_ZIP_PATH = BASE_DRIVE_PATH / "data/fairface_eval.zip"
CHALLENGE_ZIP_PATH = BASE_DRIVE_PATH / "data/challenge_dataset.zip"

# 3. Decompressione (con -o per sovrascrivere)
# Ora eseguiamo sempre l'unzip, senza controlli 'if' complicati.

print("Decompressione Pexels in corso...")
if PEXELS_ZIP_PATH.exists():
    # Aggiunto -o (overwrite) per forzare la sovrascrittura senza prompt
    !unzip -o -q "{PEXELS_ZIP_PATH}" -d "{LOCAL_DATA_DIR}"
    print("Pexels decompresso.")
else:
    print(f"ERRORE: File zip Pexels non trovato in {PEXELS_ZIP_PATH}")

print("Decompressione Pets in corso...")
if PETS_ZIP_PATH.exists():
    !unzip -o -q "{PETS_ZIP_PATH}" -d "{LOCAL_DATA_DIR}"
    print("Pets decompresso.")
else:
    print(f"ERRORE: File zip Pets non trovato in {PETS_ZIP_PATH}")

print("Decompressione FairFace in corso...")
if FAIRFACE_ZIP_PATH.exists():
    !unzip -o -q "{FAIRFACE_ZIP_PATH}" -d "{LOCAL_DATA_DIR}"
    print("FairFace decompresso.")
else:
    print(f"ERRORE: File zip FairFace non trovato in {FAIRFACE_ZIP_PATH}")

print("Decompressione Challenge Set in corso...")
if CHALLENGE_ZIP_PATH.exists():
    !unzip -o -q "{CHALLENGE_ZIP_PATH}" -d "{LOCAL_DATA_DIR}"
    print("Challenge Set decompresso.")
else:
    print(f"ERRORE: File zip Challenge Set non trovato in {CHALLENGE_ZIP_PATH}")

# 4. (Opzionale ma raccomandato) Pulizia file spazzatura Mac
print("Pulizia di eventuali file spazzatura (__MACOSX)...")
!rm -rf "{LOCAL_DATA_DIR}/__MACOSX"

print("--- Decompressione completata ---")

# -----------------------------------------------
# STEP 1.6: Correzione "in loco" del CSV Locale
# (Eseguire DOPO STEP 1.5 e PRIMA DI STEP 2)
# -----------------------------------------------
print("\n--- 1.6 Correzione Estensioni CSV sul disco locale ---")

# Questo è il path LOCALE del file CSV appena decompresso
LOCAL_CHALLENGE_CSV_PATH = CHALLENGE_DIR / "annotations_challenge.csv"

try:
    # Carica il file CSV locale
    print(f"Caricamento di {LOCAL_CHALLENGE_CSV_PATH}...")
    df = pd.read_csv(LOCAL_CHALLENGE_CSV_PATH)

    print("Correzione estensioni da .jpg a .jpeg...")

    # Controlla se le correzioni sono già state fatte
    if not df['file_name'].str.contains('.jpeg').any():
        df['file_name'] = df['file_name'].str.replace('.jpg', '.jpeg', regex=False)

        # Salva il file sovrascrivendo quello locale
        df.to_csv(LOCAL_CHALLENGE_CSV_PATH, index=False)
        print("✔️ File CSV locale corretto e salvato.")
    else:
        print("File CSV locale già corretto. Nessuna modifica necessaria.")

except FileNotFoundError:
    print(f"ERRORE: File CSV locale non trovato. Controlla i path dello STEP 1.4 e 1.5.")
except Exception as e:
    print(f"ERRORE: {e}")

print("\n--- Step 1 Completato: Ambiente pronto ---")

--- 1.1 Verifica GPU ---
Thu Oct 23 04:13:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   32C    P0             57W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+----------------------

# Dataset e Dataloading

In [2]:
# -----------------------------------------------
# STEP 2: CARICAMENTO MODELLI E DATI
# (Versione aggiornata con num_workers per velocizzare)
# -----------------------------------------------
print("\n" + "=" * 50)
print("INIZIO STEP 2: CARICAMENTO MODELLI E DATI")
print("=" * 50)

print("--- 2.1 Caricamento Modello e Processor CLIP (con Download Forzato) ---")
# (Assicurati che MODEL_ID sia "sentence-transformers/clip-resnet-50" dallo Step 1)
try:
    model = CLIPModel.from_pretrained(MODEL_ID).to(DEVICE)
    processor = CLIPProcessor.from_pretrained(MODEL_ID)
    print(f"Modello {MODEL_ID} caricato con successo su {DEVICE}.")
except Exception as e:
    print(f"Errore durante il caricamento del modello: {e}")
    import traceback
    traceback.print_exc()
print("-" * 30)

# -----------------------------------------------
# 2.2 Definizione del Dizionario Concettuale
# -----------------------------------------------
print("--- 2.2 Definizione Dizionario Concettuale ---")
CONCEPT_DICTIONARY = {
    'professions': ['engineer', 'nurse', 'doctor'],
    'gender_pairs': [
        ('man', 'woman'),
        ('male', 'female'),
        ('he', 'she'),
        ('boy', 'girl')
    ],
    'train_templates': [
        "a photo of a {}",
        "a picture of a {}",
        "an image of a {}",
        "a portrait of a {}",
        "a good photo of a {}"
    ],
    'gender_templates': [ # Non usati nel training loop, ma utili per eval
        "a photo of a {}",
        "a picture of a {}"
    ],
    'eval_templates_holdout': [
        "a high-quality image of a {}",
        "a cropped photo of a {}",
        "a {}."
    ]
}
print(f"Dizionario concettuale creato con {len(CONCEPT_DICTIONARY['professions'])} professioni.")
print("-" * 30)

# Prompt per i test su FairFace
PROMPTS_FF_GENDER = ["a photo of a man", "a photo of a woman"]
PROMPTS_FF_RACE = [
    "a photo of a White person", "a photo of a Black person", "a photo of a Latino_Hispanic person",
    "a photo of an East Asian person", "a photo of a Southeast Asian person",
    "a photo of an Indian person", "a photo of a Middle Eastern person"
]
GENDER_MAP_FF = {'Male': 0, 'Female': 1}
RACE_MAP_FF = {
    'White': 0, 'Black': 1, 'Latino_Hispanic': 2, 'East Asian': 3,
    'Southeast Asian': 4, 'Indian': 5, 'Middle Eastern': 6
}

# -----------------------------------------------
# 2.3 Caricamento Dati PEXELS (per Evaluation Bias Associazione)
# -----------------------------------------------
print("--- 2.3 Caricamento Dataset Pexels (Evaluation) ---")
class PexelsDebiasDataset(Dataset):
    def __init__(self, csv_file, img_dir, processor):
        try:
            self.annotations = pd.read_csv(csv_file)
        except FileNotFoundError:
            print(f"ERRORE: File CSV non trovato in {csv_file}")
            self.annotations = pd.DataFrame()
        self.img_dir = img_dir
        self.processor = processor
    def __len__(self): return len(self.annotations)
    def __getitem__(self, idx):
        if idx >= len(self.annotations): raise IndexError("Indice fuori range")
        row = self.annotations.iloc[idx]
        img_name = row['file_name']
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            print(f"ERRORE: Immagine non trovata: {img_path}")
            image = Image.new('RGB', (224, 224), color = 'red')
            return {
                'pixel_values': self.processor(images=image, return_tensors="pt")['pixel_values'].squeeze(),
                'profession': 'ERROR', 'gender': 'ERROR', 'file_name': img_name
            }
        processed_image = self.processor(images=image, return_tensors="pt")['pixel_values'].squeeze()
        return {
            'pixel_values': processed_image, 'profession': row['profession'],
            'gender': row['query_gender'], 'file_name': img_name
        }
try:
    pexels_dataset = PexelsDebiasDataset(PEXELS_CSV, PEXELS_IMG_DIR, processor)
    # --- ECCO LA PRIMA MODIFICA ---
    pexels_loader = DataLoader(
        pexels_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4,  # <-- MODIFICATO: Aggiunto per velocizzare
        persistent_workers=True
    )
    if len(pexels_dataset) > 0: print(f"Dataset Pexels e DataLoader creati. ({len(pexels_dataset)} immagini)")
    else: print("ATTENZIONE: Il dataset Pexels è vuoto.")
except Exception as e: print(f"Errore dataset Pexels: {e}")
print("-" * 30)

# -----------------------------------------------
# 2.4 Caricamento Dati OxfordIIITPet (per Eval Zero-Shot)
# -----------------------------------------------
print("--- 2.4 Caricamento Dataset OxfordIIITPet (Zero-Shot) ---")
try:
    # (Assicurati che l'import e il nome della classe siano corretti, es. OxfordIIITPet)
    temp_pet_data = OxfordIIITPet(root=PETS_DIR.parent, split="test", download=True)
    CLASS_NAMES_PETS = temp_pet_data.classes
    PROMPTS_PETS = [f"a photo of a {name.replace('_', ' ')}, a type of pet" for name in CLASS_NAMES_PETS]
    print(f"Nomi classi Pet e {len(PROMPTS_PETS)} prompt generati.")
except Exception as e:
    print(f"Errore nomi classi Pet: {e}")
    CLASS_NAMES_PETS, PROMPTS_PETS = [], []

class PetsZeroShotDataset(Dataset):
    def __init__(self, root_dir, processor):
        try:
            self.tv_dataset = OxfordIIITPet(root=root_dir, split="test", download=False)
        except Exception as e:
            print(f"ERRORE: Impossibile caricare OxfordIIITPet da {root_dir}. {e}")
            self.tv_dataset = []
        self.processor = processor
    def __len__(self): return len(self.tv_dataset)
    def __getitem__(self, idx):
        if idx >= len(self.tv_dataset): raise IndexError("Indice fuori range")
        image, label = self.tv_dataset[idx]
        processed_image = self.processor(images=image, return_tensors="pt")['pixel_values'].squeeze()
        return {'pixel_values': processed_image, 'label': label}
try:
    pets_dataset = PetsZeroShotDataset(PETS_DIR.parent, processor)
    # --- ECCO LA SECONDA MODIFICA ---
    pets_loader = DataLoader(
        pets_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4,  # <-- MODIFICATO: Aggiunto per velocizzare
        persistent_workers=True
    )
    if len(pets_dataset) > 0: print("Dataset Pets e DataLoader creati.")
    else: print("ATTENZIONE: Il dataset Pets è vuoto.")
except Exception as e: print(f"Errore dataset Pets: {e}")
print("-" * 30)

# -----------------------------------------------
# 2.5 NUOVO: Caricamento Dati FairFace (per Eval Representation Bias)
# -----------------------------------------------
print("--- 2.5 Caricamento Dataset FairFace (Evaluation) ---")
class FairFaceDataset(Dataset):
    def __init__(self, csv_file, img_dir, processor):
        try:
            self.annotations = pd.read_csv(csv_file)
        except FileNotFoundError:
            print(f"ERRORE: File CSV non trovato in {csv_file}")
            self.annotations = pd.DataFrame()
        self.img_dir = img_dir
        self.processor = processor
        self.annotations = self.annotations[
            self.annotations['gender'].isin(GENDER_MAP_FF.keys()) &
            self.annotations['race'].isin(RACE_MAP_FF.keys())
        ].reset_index(drop=True)
        print(f"Annotazioni filtrate: {len(self.annotations)} immagini valide.")
    def __len__(self): return len(self.annotations)
    def __getitem__(self, idx):
        if idx >= len(self.annotations): raise IndexError("Indice fuori range")
        row = self.annotations.iloc[idx]
        img_name, img_path = row['file'], os.path.join(self.img_dir, row['file'])
        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            print(f"ERRORE: Immagine non trovata: {img_path}")
            image = Image.new('RGB', (224, 224), color = 'red')
            return {'pixel_values': self.processor(images=image, return_tensors="pt")['pixel_values'].squeeze(),
                    'gender_label': -1, 'race_label': -1, 'race_group': 'ERROR'}
        processed_image = self.processor(images=image, return_tensors="pt")['pixel_values'].squeeze()
        return {
            'pixel_values': processed_image,
            'gender_label': GENDER_MAP_FF[row['gender']],
            'race_label': RACE_MAP_FF[row['race']],
            'race_group': row['race']
        }
try:
    fairface_dataset = FairFaceDataset(FAIRFACE_VAL_CSV, FAIRFACE_IMG_DIR, processor)
    # --- ECCO LA TERZA MODIFICA ---
    fairface_loader = DataLoader(
        fairface_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=4,  # <-- MODIFICATO: Aggiunto per velocizzare
        persistent_workers=True
    )
    if len(fairface_dataset) > 0: print(f"Dataset FairFace e DataLoader creati. ({len(fairface_dataset)} immagini)")
    else: print("ATTENZIONE: Il dataset FairFace è vuoto.")
except Exception as e: print(f"Errore dataset FairFace: {e}")

print("\n--- Step 2 Completato: Modelli e Tutti i Dati Caricati ---")

# 2.6 NUOVO: Caricamento Dati Challenge Set (per Eval Bias)
# -----------------------------------------------
print("--- 2.6 Caricamento Dataset Challenge Set (Evaluation) ---")
try:
    # Riusiamo la stessa classe PexelsDebiasDataset, funziona perfettamente
    challenge_dataset = PexelsDebiasDataset(
        csv_file=CHALLENGE_CSV,
        img_dir=CHALLENGE_IMG_DIR,
        processor=processor
    )

    challenge_loader = DataLoader(
        challenge_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=2,
        persistent_workers=True
    )

    if len(challenge_dataset) > 0:
        print(f"Dataset Challenge e DataLoader creati. ({len(challenge_dataset)} immagini)")
    else:
        print("ATTENZIONE: Il dataset Challenge è vuoto. Controlla i path e il file CSV.")

except Exception as e:
    print(f"Errore durante la creazione del dataset Challenge: {e}")


INIZIO STEP 2: CARICAMENTO MODELLI E DATI
--- 2.1 Caricamento Modello e Processor CLIP (con Download Forzato) ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Errore durante il caricamento del modello: openai-community/clip-rn50 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`
------------------------------
--- 2.2 Definizione Dizionario Concettuale ---
Dizionario concettuale creato con 3 professioni.
------------------------------
--- 2.3 Caricamento Dataset Pexels (Evaluation) ---
Errore dataset Pexels: name 'processor' is not defined
------------------------------
--- 2.4 Caricamento Dataset OxfordIIITPet (Zero-Shot) ---
Nomi classi Pet e 37 prompt generati.
Errore dataset Pets: name 'processor' is not defined
------------------------------
--- 2.5 Caricamento Dataset FairFace (Evaluation) ---
Errore dataset FairFace: name 'processor' is not defined

--- Step 2 Completato: Modelli e Tutti i Dati Caricati ---
--- 2.6 Caricame

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 407, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/openai-community/clip-rn50/resolve/main/config.json

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py", line 479, in cached_files
    hf_hub_download(
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py", line 114, in _inner_fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py", line 1010, in hf_hub_download
    retu

#CLIP Vanilla Analysis on Representation Bias (Fairface), Association Bias (Pexels) and Zero-shot (OxfordIIITPet)

In [ ]:
# -----------------------------------------------
# STEP 3: VALUTAZIONE INIZIALE (BASELINE)
# (MODIFICATO per includere il Challenge Set)
# -----------------------------------------------
print("\n" + "=" * 50)
print("INIZIO STEP 3: VALUTAZIONE BASELINE")
print("=" * 50)

# Dizionario per i risultati
baseline_results = {}
model.eval()

# -----------------------------------------------
# 3.1 Funzione di Valutazione: Zero-Shot (OxfordIIITPet)
# -----------------------------------------------
def evaluate_pets(model, processor, pets_loader, pets_prompts, device):
    """Esegue la classificazione zero-shot sul dataset OxfordIIITPet."""
    print("Inizio valutazione Zero-Shot (OxfordIIITPet)...")
    with torch.no_grad():
        text_inputs = processor(text=pets_prompts, return_tensors="pt", padding=True).to(device)
        text_features = model.get_text_features(**text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    all_labels, all_preds_top1, all_preds_top5 = [], [], []
    with torch.no_grad():
        for batch in tqdm(pets_loader, desc="Pets Eval"):
            images, labels = batch['pixel_values'].to(device), batch['label'].to(device)
            image_features = model.get_image_features(images)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            logits = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            top5_preds = torch.topk(logits, 5, dim=-1)
            all_labels.append(labels.cpu())
            all_preds_top1.append(top5_preds.indices[:, 0].cpu())
            all_preds_top5.append(top5_preds.indices.cpu())
    all_labels = torch.cat(all_labels).numpy()
    all_preds_top1 = torch.cat(all_preds_top1).numpy()
    all_preds_top5 = torch.cat(all_preds_top5).numpy()
    top1_accuracy = accuracy_score(all_labels, all_preds_top1)
    top5_accuracy = np.mean([all_labels[i] in all_preds_top5[i] for i in range(len(all_labels))])
    print(f"Valutazione Pets completata: Top-1={top1_accuracy:.4f}, Top-5={top5_accuracy:.4f}")
    return {"top1_accuracy": top1_accuracy, "top5_accuracy": top5_accuracy}

# -----------------------------------------------
# 3.2 Funzione di Valutazione: Association Bias (Pexels / Challenge)
# -----------------------------------------------
def evaluate_pexels_bias(model, processor, pexels_loader, concept_dict, device,
                         eval_template="a photo of a {} {}"):
    """
    Calcola il bias di associazione (professione-genere).
    (Questa funzione è generica e funziona sia per Pexels che per Challenge Set)
    """
    print(f"Inizio valutazione Association Bias con template: '{eval_template}'...")
    professions = concept_dict['professions']
    eval_prompts, prompt_map = [], {}

    for prof in professions:
        for gender in ['male', 'female']:
            prompt = eval_template.format(gender, prof)
            prompt_map[(prof, gender)] = len(eval_prompts)
            eval_prompts.append(prompt)

    with torch.no_grad():
        text_inputs = processor(text=eval_prompts, return_tensors="pt", padding=True).to(device)
        text_features = model.get_text_features(**text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    results = []
    with torch.no_grad():
        # Usiamo desc="Eval" per renderlo generico
        for batch in tqdm(pexels_loader, desc="Association Bias Eval"):
            images = batch['pixel_values'].to(device)
            image_features = model.get_image_features(images)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            logits = image_features @ text_features.T

            for i in range(len(batch['profession'])):
                prof, gender_gt = batch['profession'][i], batch['gender'][i]
                if prof == 'ERROR': continue
                sim_male_prompt = logits[i, prompt_map[(prof, 'male')]].item()
                sim_female_prompt = logits[i, prompt_map[(prof, 'female')]].item()
                results.append({
                    "profession": prof, "img_gender": gender_gt,
                    "sim_male_prompt": sim_male_prompt, "sim_female_prompt": sim_female_prompt,
                    "bias_score_diff": sim_female_prompt - sim_male_prompt
                })

    df = pd.DataFrame(results)

    # Gestione del caso in cui il dataset è vuoto
    if df.empty:
        print("Valutazione completata. ATTENZIONE: Nessun dato trovato nel loader.")
        return {"grouped_bias": pd.Series(), "overall_abs_bias": 0.0}

    bias_summary = df.groupby(['profession', 'img_gender'])['bias_score_diff'].mean()
    overall_abs_bias = df['bias_score_diff'].abs().mean()

    print("Valutazione Association Bias completata.")
    print("Bias raggruppato (Mean diff):")
    print(bias_summary)
    print(f"\nOverall Absolute Bias Score: {overall_abs_bias:.4f}")

    return {"grouped_bias": bias_summary, "overall_abs_bias": overall_abs_bias}

# -----------------------------------------------
# 3.3 Funzione di Valutazione: Representation Bias (FairFace)
# -----------------------------------------------
def evaluate_fairface_bias(model, processor, fairface_loader, gender_prompts, race_prompts, device):
    """Valuta l'accuratezza della classificazione di genere e etnia su FairFace."""
    print("Inizio valutazione Representation Bias (FairFace)...")
    with torch.no_grad():
        text_inputs_gender = processor(text=gender_prompts, return_tensors="pt", padding=True).to(device)
        text_features_gender = model.get_text_features(**text_inputs_gender)
        text_features_gender /= text_features_gender.norm(dim=-1, keepdim=True)
        text_inputs_race = processor(text=race_prompts, return_tensors="pt", padding=True).to(device)
        text_features_race = model.get_text_features(**text_inputs_race)
        text_features_race /= text_features_race.norm(dim=-1, keepdim=True)

    results = []
    with torch.no_grad():
        for batch in tqdm(fairface_loader, desc="FairFace Eval"):
            images = batch['pixel_values'].to(device)
            g_labels, r_labels = batch['gender_label'].to(device), batch['race_label'].to(device)
            r_groups = batch['race_group']

            image_features = model.get_image_features(images)
            image_features /= image_features.norm(dim=-1, keepdim=True)

            preds_gender = (100.0 * image_features @ text_features_gender.T).argmax(dim=-1)
            preds_race = (100.0 * image_features @ text_features_race.T).argmax(dim=-1)

            for i in range(len(r_groups)):
                results.append({
                    "race_group": r_groups[i],
                    "gender_correct": (preds_gender[i] == g_labels[i]).item(),
                    "race_correct": (preds_race[i] == r_labels[i]).item(),
                })

    df = pd.DataFrame(results)

    if df.empty:
        print("Valutazione FairFace completata. ATTENZIONE: Nessun dato trovato.")
        return {
            "overall_gender_acc": 0.0, "overall_race_acc": 0.0,
            "gender_acc_by_race": pd.Series(), "race_acc_by_race": pd.Series()
        }

    overall_gender_acc = df['gender_correct'].mean()
    overall_race_acc = df['race_correct'].mean()
    gender_acc_by_race = df.groupby('race_group')['gender_correct'].mean().sort_values()
    race_acc_by_race = df.groupby('race_group')['race_correct'].mean().sort_values()

    print("Valutazione FairFace completata.")
    print(f"Accuratezza Genere (Overall): {overall_gender_acc:.4f}")
    print(f"Accuratezza Etnia (Overall): {overall_race_acc:.4f}")
    print("\nAccuratezza Genere per Gruppo Etnico:")
    print(gender_acc_by_race)

    return {
        "overall_gender_acc": overall_gender_acc, "overall_race_acc": overall_race_acc,
        "gender_acc_by_race": gender_acc_by_race, "race_acc_by_race": race_acc_by_race
    }

# -----------------------------------------------
# 3.4 Funzione di Plotting: PCA dello Spazio Testuale
# -----------------------------------------------
def plot_text_pca(model, processor, concept_dict, device, title, save_path=None):
    """Crea un plot PCA 2D degli embedding testuali."""
    print("Inizio analisi PCA (Spazio Testuale)...")
    prof_labels = concept_dict['professions']
    gender_labels = ['man', 'woman', 'male', 'female']
    all_labels = prof_labels + gender_labels
    prompts = [f"a photo of a {label}" for label in all_labels]

    with torch.no_grad():
        text_inputs = processor(text=prompts, return_tensors="pt", padding=True).to(device)
        text_features = model.get_text_features(**text_inputs)
        text_features_norm = text_features / text_features.norm(dim=-1, keepdim=True)

    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(text_features_norm.cpu().numpy())

    df = pd.DataFrame(embeddings_2d, columns=['PC1', 'PC2'])
    df['label'] = all_labels
    df['type'] = ['Profession'] * len(prof_labels) + ['Gender'] * len(gender_labels)

    plt.figure(figsize=(10, 7))
    sns.scatterplot(data=df, x='PC1', y='PC2', hue='type', style='type', s=200)
    for i, row in df.iterrows():
        plt.text(row['PC1'] + 0.01, row['PC2'], row['label'], fontsize=9)
    plt.title(title)
    plt.xlabel("Componente Principale 1"); plt.ylabel("Componente Principale 2")
    plt.grid(True, linestyle='--', alpha=0.6)

    if save_path:
        try:
            plt.savefig(save_path, bbox_inches='tight')
            print(f"Grafico PCA (Testo) salvato in: {save_path}")
        except Exception as e:
            print(f"Errore durante il salvataggio del grafico PCA: {e}")

    plt.show()
    print("Plot PCA (Testo) generato.")

# -----------------------------------------------
# 3.5 Funzione di Plotting: PCA dello Spazio Immagini (FairFace)
# -----------------------------------------------
def plot_image_pca(model, fairface_loader, device, title_prefix):
    """Crea un plot PCA 2D degli embedding delle immagini di FairFace."""
    print("Inizio analisi PCA (Spazio Immagini - FairFace)...")
    all_features, all_gender_labels, all_race_groups = [], [], []
    with torch.no_grad():
        # Usiamo un subset per velocizzare la PCA (es. max 5000 immagini)
        count = 0
        max_images_for_pca = 5000

        for batch in tqdm(fairface_loader, desc="PCA Immagini"):
            images = batch['pixel_values'].to(device)
            image_features = model.get_image_features(images)
            image_features_norm = image_features / image_features.norm(dim=-1, keepdim=True)

            all_features.append(image_features_norm.cpu())
            all_gender_labels.extend(batch['gender_label'].numpy())
            all_race_groups.extend(batch['race_group'])

            count += len(images)
            if count > max_images_for_pca:
                print(f"Raggiunto limite di {max_images_for_pca} immagini per PCA. Interrompo.")
                break

    all_features = torch.cat(all_features).numpy()

    if all_features.shape[0] == 0:
        print("Nessuna immagine trovata per la PCA. Salto il plot.")
        return

    pca = PCA(n_components=2)
    features_2d = pca.fit_transform(all_features)

    df = pd.DataFrame(features_2d, columns=['PC1', 'PC2'])
    df['gender'] = ['Female' if g == 1 else 'Male' for g in all_gender_labels]
    df['race'] = all_race_groups

    # Non c'è più bisogno di campionare, lo abbiamo fatto prima
    # if len(df) > 2000: df = df.sample(n=2000, random_state=42)

    plt.figure(figsize=(10, 7))
    sns.scatterplot(data=df, x='PC1', y='PC2', hue='gender', alpha=0.6, s=50)
    plt.title(f"{title_prefix} - PCA Spazio Immagini (Colorato per Genere)")
    plt.grid(True, linestyle='--', alpha=0.6); plt.show()

    plt.figure(figsize=(12, 8))
    sns.scatterplot(data=df, x='PC1', y='PC2', hue='race', alpha=0.6, s=50)
    plt.title(f"{title_prefix} - PCA Spazio Immagini (Colorato per Etnia)")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2); plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout(); plt.show()
    print("Plot PCA (Immagini) generati.")

# -----------------------------------------------
# 3.6 Esecuzione della Valutazione Baseline
# -----------------------------------------------

try:
    with torch.no_grad():
        # Test 1: Performance Zero-Shot
        print("\n--- 3.6.1 Esecuzione Test Zero-Shot (Pets) ---")
        baseline_results["pets"] = evaluate_pets(
            model, processor, pets_loader, PROMPTS_PETS, DEVICE
        )

        # Test 2: Bias di Associazione (Professioni - Pexels Standard)
        print("\n--- 3.6.2 Esecuzione Test Bias Associazione (Pexels) ---")
        baseline_results["pexels_bias"] = evaluate_pexels_bias(
            model, processor, pexels_loader, CONCEPT_DICTIONARY, DEVICE
        )

        # Test 3: Bias di Rappresentazione (Genere/Etnia - FairFace)
        print("\n--- 3.6.3 Esecuzione Test Rappresentazione (FairFace) ---")
        baseline_results["fairface_bias"] = evaluate_fairface_bias(
            model, processor, fairface_loader, PROMPTS_FF_GENDER, PROMPTS_FF_RACE, DEVICE
        )

        # --- BLOCCO AGGIUNTO ---
        print("\n--- 3.6.4 Esecuzione Test Challenge Set (Baseline) ---")
        # (Usa il challenge_loader definito nello STEP 2.6)
        baseline_results["challenge_bias"] = evaluate_pexels_bias(
            model, processor, challenge_loader, CONCEPT_DICTIONARY, DEVICE
        )
        # --- FINE BLOCCO AGGIUNTO ---

    # Test 4: PCA Spazio Testuale (con salvataggio)
    print("\n--- 3.6.5 Esecuzione PCA Spazio Testuale ---")
    plot_text_pca(
        model, processor, CONCEPT_DICTIONARY, DEVICE,
        title="Baseline: PCA Spazio Testuale",
        save_path=OUTPUT_DIR / "baseline_pca_text.png"
    )

    # Test 5: PCA Spazio Immagini (Non serve salvarla, è statica)
    print("\n--- 3.6.6 Esecuzione PCA Spazio Immagini ---")
    plot_image_pca(
        model, fairface_loader, DEVICE,
        title_prefix="Baseline"
    )

except Exception as e:
    print(f"\nERRORE DURANTE LA VALUTAZIONE BASELINE: {e}")
    import traceback
    traceback.print_exc()

print("\n--- Step 3 Completato: Valutazione Baseline Eseguita ---")
print("\nRiepilogo Risultati Baseline:")

# -----------------------------------------------
# 3.7 Funzione di Conversione e Salvataggio Risultati
# -----------------------------------------------

# (Questa è la funzione "intelligente" che gestisce tutte le chiavi)
def convert_results_to_json_serializable(results):
    """
    Converte i risultati dell'esperimento (che contengono oggetti pandas)
    in un dizionario serializzabile in JSON.
    Gestisce automaticamente tutte le chiavi di tipo 'bias'.
    """
    results_copy = results.copy()

    # Lista di chiavi che hanno lo stesso formato di 'pexels_bias'
    pexels_style_keys = ['pexels_bias', 'pexels_bias_holdout', 'challenge_bias']

    for key in pexels_style_keys:
        if key in results_copy and results_copy[key] is not None:
            # Controllo per assicurarsi che 'grouped_bias' non sia una Serie vuota
            if not results_copy[key]['grouped_bias'].empty:
                grouped_data = results_copy[key]['grouped_bias'].reset_index().to_dict('records')
            else:
                grouped_data = [] # Salva come lista vuota

            results_copy[key] = {
                'grouped_bias_records': grouped_data,
                'overall_abs_bias': results_copy[key]['overall_abs_bias']
            }

    # Gestione FairFace (resta invariato)
    if 'fairface_bias' in results_copy and results_copy['fairface_bias'] is not None:
        results_copy['fairface_bias'] = results_copy['fairface_bias'].copy()
        if 'gender_acc_by_race' in results_copy['fairface_bias'] and not results_copy['fairface_bias']['gender_acc_by_race'].empty:
            results_copy['fairface_bias']['gender_acc_by_race'] = results_copy['fairface_bias']['gender_acc_by_race'].to_dict()
        else:
            results_copy['fairface_bias']['gender_acc_by_race'] = {} # Salva come dizionario vuoto

        if 'race_acc_by_race' in results_copy['fairface_bias'] and not results_copy['fairface_bias']['race_acc_by_race'].empty:
            results_copy['fairface_bias']['race_acc_by_race'] = results_copy['fairface_bias']['race_acc_by_race'].to_dict()
        else:
             results_copy['fairface_bias']['race_acc_by_race'] = {} # Salva come dizionario vuoto

    return results_copy

# Stampa e salva i risultati
serializable_baseline_results = convert_results_to_json_serializable(baseline_results)
print(json.dumps(serializable_baseline_results, indent=2))

print("\n--- 3.7 Salvataggio Risultati Baseline su Drive ---")
baseline_save_path = OUTPUT_DIR / "baseline_results.json"
try:
    # MODIFICA: Aggiungiamo 'challenge_bias' al dizionario
    results_to_save = {
        "pets": serializable_baseline_results.get("pets", {}),
        "fairface_bias": serializable_baseline_results.get("fairface_bias", {}),
        "pexels_bias": serializable_baseline_results.get("pexels_bias", {}),
        "challenge_bias": serializable_baseline_results.get("challenge_bias", {}) # <-- RIGA AGGIUNTA
    }
    with open(baseline_save_path, 'w') as f:
        json.dump(results_to_save, f, indent=4)
    print(f"Risultati Pets, FairFace, Pexels e Challenge (Baseline) salvati in: {baseline_save_path}")
except Exception as e:
    print(f"Errore durante il salvataggio dei risultati baseline: {e}")


INIZIO STEP 3: VALUTAZIONE BASELINE


NameError: name 'model' is not defined

# FineTuning Setup
- LoRa
- Loss Definition
- Utilis

In [ ]:
# -----------------------------------------------
# STEP 4: PREPARAZIONE PER IL FINETUNING
# (NUOVO CODICE)
# -----------------------------------------------
print("\n" + "=" * 50)
print("INIZIO STEP 4: PREPARAZIONE FINETUNING")
print("=" * 50)

# -----------------------------------------------
# 4.1 Calcolo degli "Ancoraggi" (Mean Embeddings)
# (Come da nostra discussione)
# -----------------------------------------------
def calculate_anchors(model, processor, concept_dict, device):
    """
    Calcola gli embedding di ancoraggio come media dei template di training.
    """
    print("Inizio calcolo Ancoraggi (Mean Embeddings)...")
    model.eval() # Assicurati di essere in modalità eval
    anchors = {}

    with torch.no_grad():
        for prof in concept_dict['professions']:
            # 1. Crea tutti i prompt di training per questa professione
            prompts = [template.format(prof) for template in concept_dict['train_templates']]

            # 2. Codifica tutti i prompt in un batch
            inputs = processor(text=prompts, return_tensors="pt", padding=True).to(device)
            text_features = model.get_text_features(**inputs)

            # 3. Normalizza gli embedding
            text_features_norm = text_features / text_features.norm(dim=-1, keepdim=True)

            # 4. Calcola la media e salva
            mean_anchor = text_features_norm.mean(dim=0)

            # 5. Normalizza di nuovo la media (importante!)
            anchors[prof] = mean_anchor / mean_anchor.norm()

            print(f"Ancoraggio calcolato per: {prof}")

    print("Calcolo ancoraggi completato.")
    return anchors

# Calcoliamo gli ancoraggi usando il modello *originale*
anchors = calculate_anchors(model, processor, CONCEPT_DICTIONARY, DEVICE)
print("-" * 30)

# -----------------------------------------------
# 4.2 Configurazione di LoRA (PEFT)
# (Come da nostra discussione)
# -----------------------------------------------
print("Configurazione di LoRA (PEFT) sul Text Encoder...")

# 1. Definiamo la configurazione LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    # Applichiamo LoRA a tutti i layer di proiezione dell'Attention
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
)

# 2. Congeliamo esplicitamente il Vision Transformer (buona prassi)
for param in model.vision_model.parameters():
    param.requires_grad = False
print("Vision Transformer (ViT) congelato.")

# 3. Applichiamo LoRA *solo* al Text Encoder
model.text_model = get_peft_model(model.text_model, lora_config)
print("LoRA applicato al Text Encoder.")

# 4. Stampa i parametri addestrabili per conferma
model.text_model.print_trainable_parameters() # <-- CORRETTO
print("-" * 30)

# -----------------------------------------------
# 4.3 Definizione Iperparametri e Ottimizzatore
# -----------------------------------------------

# Iperparametri
NUM_STEPS = 1500
LEARNING_RATE = 1e-5  # Come da piano (1e-5 o 5e-6)
LAMBDA_ANCHOR = 0.05   # Peso della anchor loss (critico)
DEBIAS_MARGIN = 0.1   # Margine per la L_debias (come da accordo)

# Definiamo l'ottimizzatore
# Passiamo solo i parametri del modello. PEFT si occupa di
# restituire solo quelli addestrabili (i pesi LoRA).
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

print(f"Iperparametri impostati:")
print(f"  - STEPS: {NUM_STEPS}")
print(f"  - LR: {LEARNING_RATE}")
print(f"  - LAMBDA_ANCHOR: {LAMBDA_ANCHOR}")
print(f"  - DEBIAS_MARGIN: {DEBIAS_MARGIN}")
print("Ottimizzatore AdamW creato.")
# -----------------------------------------------
# 4.4 Tentativo di Caricamento Pesi Esistenti
# (NUOVO BLOCCO AGGIUNTO)
# -----------------------------------------------
print("\n--- 4.4 Definizione Path e Tentativo di Caricamento Adattatori ---")

# Definiamo il path qui, così è disponibile sia per il loading (ora)
# che per il saving (nello Step 5)
ADAPTER_SAVE_PATH = OUTPUT_DIR / "finetuned_clip_lora_adapters"

skip_training = False # Flag per saltare il training se i pesi esistono

try:
    # Prova a caricare i pesi.
    # Applichiamo i pesi LoRA al text_model, perché è quello che
    # abbiamo configurato per il training
    model.text_model = PeftModel.from_pretrained(model.text_model, ADAPTER_SAVE_PATH)

    # Se il caricamento riesce:
    skip_training = True
    model.eval() # Mettiamo il modello in modalità valutazione
    print(f"!!! SUCCESSO: Pesi LoRA pre-addestrati trovati e caricati da: {ADAPTER_SAVE_PATH}")
    print("--- Verrà saltato il TRAINING (STEP 5) ---")

except Exception as e:
    # L'eccezione più comune è OSError se la cartella non esiste
    print(f"Pesi LoRA non trovati in {ADAPTER_SAVE_PATH} (Errore: {e})")
    print("Si procederà con il training normale (STEP 5).")

print("\n--- Step 4 Completato: Preparazione Finetuning ---")

# TRAINING LOOP EXECUTION


In [ ]:
# -----------------------------------------------
# STEP 5: ESECUZIONE DEL FINETUNING
# (MODIFICATO per saltare se i pesi sono stati caricati)
# -----------------------------------------------
print("\n" + "=" * 50)
print("INIZIO STEP 5: ESECUZIONE FINETUNING")
print("=" * 50)

# Inizializziamo la history qui, così esiste in ogni caso
loss_history = {
    'total': [],
    'debias': [],
    'anchor': []
}

# Controlliamo il flag impostato nello STEP 4
# La variabile 'skip_training' e 'ADAPTER_SAVE_PATH' dovrebbero essere
# state definite alla fine dello STEP 4
if skip_training:
    print("STEP 5 SKIPPATO: I pesi del modello sono già stati caricati.")

else:
    # --- Se i pesi non sono stati trovati, esegui il training ---
    print("Avvio del training loop...")

    # Mettiamo il modello in modalità training (attiva i dropout LoRA)
    model.train()

    # Iniziamo il training loop
    pbar = tqdm(range(NUM_STEPS), desc="Finetuning Steps")
    for step in pbar:

        # --- 1. Campionamento Casuale ---
        prof = random.choice(CONCEPT_DICTIONARY['professions'])
        g_pair = random.choice(CONCEPT_DICTIONARY['gender_pairs'])
        template = random.choice(CONCEPT_DICTIONARY['train_templates'])

        # --- 2. Preparazione Dati per lo Step ---
        # Prendiamo l'ancoraggio pre-calcolato
        anchor_embed = anchors[prof].to(DEVICE)

        # Creiamo i prompt
        prompt_p = template.format(prof)
        prompt_g1 = template.format(g_pair[0]) # es. "a photo of a man"
        prompt_g2 = template.format(g_pair[1]) # es. "a photo of a woman"

        # --- 3. Calcolo Embedding Correnti ---
        # Codifichiamo tutto in un unico batch
        inputs = processor(
            text=[prompt_p, prompt_g1, prompt_g2],
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(DEVICE)

        # Otteniamo gli embedding dal text encoder (che ora ha LoRA)
        text_features = model.get_text_features(**inputs)
        text_features_norm = text_features / text_features.norm(dim=-1, keepdim=True)

        # Separiamo gli embedding
        current_embed_p = text_features_norm[0]
        embed_g1 = text_features_norm[1]
        embed_g2 = text_features_norm[2]

        # --- 4. Calcolo Loss Function Doppia ---

        # L_debias (Equidistanza con Margin Loss)
        sim_p_g1 = cosine_similarity(current_embed_p.unsqueeze(0), embed_g1.unsqueeze(0)).squeeze()
        sim_p_g2 = cosine_similarity(current_embed_p.unsqueeze(0), embed_g2.unsqueeze(0)).squeeze()

        # L_debias = max(0, abs(sim1 - sim2) - MARGIN)^2
        L_debias = (sim_p_g1 - sim_p_g2)**2

        # L_anchor (Anti-Drift)
        # Vogliamo massimizzare la similarità, quindi minimizziamo (1 - similarità)
        L_anchor = 1.0 - cosine_similarity(current_embed_p.unsqueeze(0), anchor_embed.unsqueeze(0)).squeeze()

        # L_total
        L_total = L_debias + LAMBDA_ANCHOR * L_anchor

        # --- 5. Backpropagation ---
        optimizer.zero_grad()
        L_total.backward()
        optimizer.step()

        # --- 6. Logging ---
        loss_history['total'].append(L_total.item())
        loss_history['debias'].append(L_debias.item())
        loss_history['anchor'].append(L_anchor.item())

        if step % 100 == 0 or step == NUM_STEPS - 1:
            avg_total = np.mean(loss_history['total'][-100:])
            avg_debias = np.mean(loss_history['debias'][-100:])
            avg_anchor = np.mean(loss_history['anchor'][-100:])
            pbar.set_description(
                f"Step {step} | L_total: {avg_total:.4f} (L_debias: {avg_debias:.4f}, L_anchor: {avg_anchor:.4f})"
            )

    print("Finetuning completato.")

    # Plot delle loss
    plt.figure(figsize=(12, 5))
    plt.plot(loss_history['total'], label='L_total', alpha=0.8)
    plt.plot(loss_history['debias'], label='L_debias', linestyle='--')
    plt.plot(loss_history['anchor'], label=f'L_anchor (x{LAMBDA_ANCHOR})', linestyle=':')
    plt.title("Andamento delle Loss durante il Training")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    # --- BLOCCO SALVATAGGIO GRAFICO LOSS ---
    loss_plot_path = OUTPUT_DIR / "finetuning_loss_history.png"
    try:
        plt.savefig(loss_plot_path, bbox_inches='tight')
        print(f"Grafico Loss salvato in: {loss_plot_path}")
    except Exception as e:
        print(f"Errore salvataggio grafico loss: {e}")
    # --- FINE BLOCCO ---

    plt.show()

    # --- BLOCCO SALVATAGGIO DATI LOSS (JSON) ---
    print("\n--- 5.1 Salvataggio Dati Loss History su Drive ---")
    loss_data_path = OUTPUT_DIR / "finetuning_loss_history.json"
    try:
        with open(loss_data_path, 'w') as f:
            json.dump(loss_history, f, indent=4)
        print(f"Dati della Loss History salvati in: {loss_data_path}")
    except Exception as e:
        print(f"Errore durante il salvataggio dei dati di loss: {e}")
    # --- FINE BLOCCO ---

    # --- BLOCCO SALVATAGGIO PESI MODELLO (ADATTATORI LORA) ---
    print("\n--- 5.2 Salvataggio Pesi del Modello (Adattatori LoRA) ---")
    # (Usiamo ADAPTER_SAVE_PATH definito nello Step 4.4)
    try:
        # Salviamo solo i pesi addestrati (adattatori LoRA)
        model.save_pretrained(ADAPTER_SAVE_PATH)

        # Salviamo anche la config del text_model per coerenza
        model.config.text_config.save_pretrained(ADAPTER_SAVE_PATH)

        print(f"Adattatori LoRA salvati con successo in: {ADAPTER_SAVE_PATH}")
    except Exception as e:
        print(f"Errore during il salvataggio degli adattatori del modello: {e}")
    # --- FINE BLOCCO ---

print("\n--- Step 5 Completato ---")

# POST FINE-TUNING EVALUATION

In [ ]:
# -----------------------------------------------
# STEP 6: VALUTAZIONE FINALE (POST-FINETUNING)
# (MODIFICATO per includere il Challenge Set)
# -----------------------------------------------
print("\n" + "=" * 50)
print("INIZIO STEP 6: VALUTAZIONE POST-FINETUNING")
print("=" * 50)

post_results = {}
model.eval()

try:
    with torch.no_grad():

        # --- Test 1: Performance Zero-Shot (Pets) ---
        print("\n--- 6.1 Esecuzione Test Zero-Shot (Pets) Post-Finetuning ---")
        post_results["pets"] = evaluate_pets(
            model, processor, pets_loader, PROMPTS_PETS, DEVICE
        )

        # --- Test 2: Bias di Associazione (Pexels - Template Standard) ---
        print("\n--- 6.2 Esecuzione Test Bias Associazione (Pexels) Post-Finetuning ---")
        post_results["pexels_bias"] = evaluate_pexels_bias(
            model, processor, pexels_loader, CONCEPT_DICTIONARY, DEVICE,
            eval_template="a photo of a {} {}"
        )

        # --- Test 3: Bias di Rappresentazione (FairFace) ---
        print("\n--- 6.3 Esecuzione Test Rappresentazione (FairFace) Post-Finetuning ---")
        post_results["fairface_bias"] = evaluate_fairface_bias(
            model, processor, fairface_loader, PROMPTS_FF_GENDER, PROMPTS_FF_RACE, DEVICE
        )

        # --- Test 4: Generalizzazione (Pexels - Template Hold-out) ---
        print("\n--- 6.4 Esecuzione Test Generalizzazione (Pexels Hold-out) Post-Finetuning ---")
        holdout_template = "a high-quality image of a {} {}"
        post_results["pexels_bias_holdout"] = evaluate_pexels_bias(
            model, processor, pexels_loader, CONCEPT_DICTIONARY, DEVICE,
            eval_template=holdout_template
        )

        # --- BLOCCO AGGIUNTO ---
        print("\n--- 6.5 Esecuzione Test Challenge Set (Post-Finetuning) ---")
        # (Usa il challenge_loader definito nello STEP 2.6)
        post_results["challenge_bias"] = evaluate_pexels_bias(
            model, processor, challenge_loader, CONCEPT_DICTIONARY, DEVICE
        )
        # --- FINE BLOCCO AGGIUNTO ---

    # --- Test 6: PCA Spazio Testuale (Post) (con salvataggio) ---
    print("\n--- 6.6 Esecuzione PCA Spazio Testuale Post-Finetuning ---")
    plot_text_pca(
        model, processor, CONCEPT_DICTIONARY, DEVICE,
        title="Post-Finetuning: PCA Spazio Testuale",
        save_path=OUTPUT_DIR / "post_finetuning_pca_text.png"
    )

except Exception as e:
    print(f"\nERRORE DURANTE LA VALUTAZIONE POST-FINETUNING: {e}")
    import traceback
    traceback.print_exc()

print("\n--- Step 6 Completato: Valutazione Post-Finetuning Eseguita ---")
print("\nRiepilogo Risultati Post-Finetuning:")

# (Assicurati che la funzione 'convert_results_to_json_serializable' sia definita nello Step 3)
serializable_post_results = convert_results_to_json_serializable(post_results)
print(json.dumps(serializable_post_results, indent=2))

# --- 6.7 Salvataggio Risultati Post-Finetuning su Drive ---

print("\n--- 6.7 Salvataggio Risultati Post-Finetuning su Drive ---")
post_save_path = OUTPUT_DIR / "post_finetuning_results.json"
try:
    # MODIFICA: Aggiungiamo 'challenge_bias'
    results_to_save = {
        "pets": serializable_post_results.get("pets", {}),
        "fairface_bias": serializable_post_results.get("fairface_bias", {}),
        "pexels_bias": serializable_post_results.get("pexels_bias", {}),
        "pexels_bias_holdout": serializable_post_results.get("pexels_bias_holdout", {}),
        "challenge_bias": serializable_post_results.get("challenge_bias", {}) # <-- RIGA AGGIUNTA
    }
    with open(post_save_path, 'w') as f:
        json.dump(results_to_save, f, indent=4)
    print(f"Risultati Pets, FairFace, Pexels e Challenge (Post-Finetuning) salvati in: {post_save_path}")
except Exception as e:
    print(f"Errore durante il salvataggio dei risultati post-finetuning: {e}")

# Comparation PRE POST FINE-TUNING

In [ ]:
# -----------------------------------------------
# STEP 7: ANALISI COMPARATIVA E CONCLUSIONI
# (MODIFICATO per includere il Challenge Set ovunque)
# -----------------------------------------------
print("\n" + "=" * 50)
print("INIZIO STEP 7: ANALISI COMPARATIVA (PRE vs POST)")
print("=" * 50)

# -----------------------------------------------
# 7.1 Confronto Performance Zero-Shot (OxfordIIITPet)
# -----------------------------------------------
print("\n--- 7.1 Confronto Performance Zero-Shot (Pets) ---")
try:
    pets_data = {
        'Baseline': baseline_results['pets'],
        'Post-Finetune': post_results['pets']
    }
    pets_df = pd.DataFrame(pets_data).T
    print(pets_df)

    pets_df.plot(
        kind='bar',
        title='Confronto Accuratezza Zero-Shot (OxfordIIITPet)',
        figsize=(8, 5)
    )
    plt.ylabel('Accuratezza')
    plt.xticks(rotation=0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.ylim(bottom=max(0, pets_df.min().min() - 0.1)) # Imposta un limite y ragionevole
    plt.savefig(OUTPUT_DIR / "comparison_pets_accuracy.png", bbox_inches='tight')
    print(f"Grafico Pets salvato in: {OUTPUT_DIR / 'comparison_pets_accuracy.png'}")
    plt.show()
except KeyError:
    print("Dati 'pets' non trovati. Salto del grafico.")

# -----------------------------------------------
# 7.2 Confronto Bias di Associazione (Pexels)
# -----------------------------------------------
print("\n--- 7.2 Confronto Bias di Associazione (Pexels) ---")
try:
    bias_data = {
        'Baseline (Template Standard)': baseline_results['pexels_bias']['overall_abs_bias'],
        'Post-Finetune (Template Standard)': post_results['pexels_bias']['overall_abs_bias'],
        'Post-Finetune (Template Hold-out)': post_results['pexels_bias_holdout']['overall_abs_bias']
    }
    bias_df = pd.DataFrame.from_dict(bias_data, orient='index', columns=['Overall Absolute Bias'])
    print(bias_df)

    bias_df.plot(
        kind='bar',
        title='Confronto Bias di Associazione (Pexels)',
        figsize=(10, 6),
        legend=False
    )
    plt.ylabel('Overall Absolute Bias Score (Vicino a 0 è meglio)')
    plt.xticks(rotation=15, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(OUTPUT_DIR / "comparison_pexels_bias.png", bbox_inches='tight')
    print(f"Grafico Pexels Bias salvato in: {OUTPUT_DIR / 'comparison_pexels_bias.png'}")
    plt.show()
except KeyError:
    print("Dati 'pexels_bias' non trovati. Salto del grafico.")

# -----------------------------------------------
# 7.3 Confronto Bias di Rappresentazione (FairFace)
# -----------------------------------------------
print("\n--- 7.3 Confronto Bias di Rappresentazione (FairFace) ---")
try:
    baseline_ff = baseline_results['fairface_bias']['gender_acc_by_race']
    post_ff = post_results['fairface_bias']['gender_acc_by_race']

    ff_df = pd.DataFrame({
        'Baseline': baseline_ff,
        'Post-Finetune': post_ff
    })
    print(ff_df)

    ff_df.plot(
        kind='bar',
        title='Accuratezza Classificazione Genere per Etnia (FairFace)',
        figsize=(12, 7)
    )
    plt.ylabel('Accuratezza')
    plt.xlabel('Gruppo Etnico (FairFace)')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.ylim(bottom=max(0, ff_df.min().min() - 0.1))
    plt.savefig(OUTPUT_DIR / "comparison_fairface_accuracy.png", bbox_inches='tight')
    print(f"Grafico FairFace Accuracy salvato in: {OUTPUT_DIR / 'comparison_fairface_accuracy.png'}")
    plt.show()
except KeyError:
    print("Dati 'fairface_bias' non trovati. Salto del grafico.")

# -----------------------------------------------
# 7.4 Confronto Association Bias (Challenge Set)
# -----------------------------------------------
print("\n--- 7.4 Confronto Bias di Associazione (Challenge Set) ---")

try:
    challenge_bias_data = {
        'Baseline (Challenge Set)': baseline_results['challenge_bias']['overall_abs_bias'],
        'Post-Finetune (Challenge Set)': post_results['challenge_bias']['overall_abs_bias']
    }
    challenge_bias_df = pd.DataFrame.from_dict(challenge_bias_data, orient='index', columns=['Overall Absolute Bias'])
    print(challenge_bias_df)

    challenge_bias_df.plot(
        kind='bar',
        title='Confronto Bias di Associazione (Challenge Set Ambigue)',
        figsize=(8, 5),
        legend=False
    )
    plt.ylabel('Overall Absolute Bias Score (Vicino a 0 è meglio)')
    plt.xticks(rotation=0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(OUTPUT_DIR / "comparison_challenge_bias.png", bbox_inches='tight')
    print(f"Grafico Challenge Bias salvato in: {OUTPUT_DIR / 'comparison_challenge_bias.png'}")
    plt.show()

except KeyError:
    print("Dati del 'challenge_bias' non trovati. Salto del grafico.")


# -----------------------------------------------
# 7.5 Conclusioni
# -----------------------------------------------
print("\n--- 7.5 Conclusioni ---")
# Calcoliamo le differenze
try:
    # Pets
    pet_top1_diff = post_results['pets']['top1_accuracy'] - baseline_results['pets']['top1_accuracy']

    # Pexels
    bias_diff = post_results['pexels_bias']['overall_abs_bias'] - baseline_results['pexels_bias']['overall_abs_bias']
    if baseline_results['pexels_bias']['overall_abs_bias'] > 0:
        bias_perc_reduction = (-bias_diff / baseline_results['pexels_bias']['overall_abs_bias']) * 100
    else:
        bias_perc_reduction = float('inf') if bias_diff < 0 else 0.0

    # FairFace
    ff_gender_diff = post_results['fairface_bias']['overall_gender_acc'] - baseline_results['fairface_bias']['overall_gender_acc']

    # --- BLOCCO AGGIUNTO ---
    # Challenge Set
    challenge_bias_diff = post_results['challenge_bias']['overall_abs_bias'] - baseline_results['challenge_bias']['overall_abs_bias']
    if baseline_results['challenge_bias']['overall_abs_bias'] > 0:
        challenge_bias_perc_reduction = (-challenge_bias_diff / baseline_results['challenge_bias']['overall_abs_bias']) * 100
    else:
        challenge_bias_perc_reduction = float('inf') if challenge_bias_diff < 0 else 0.0
    # --- FINE BLOCCO AGGIUNTO ---

    print("Analisi Quantitativa (PRE vs POST):")
    print("-" * 30)

    print(f"🎯 Performance Zero-Shot (Pets):")
    print(f"  - Baseline Top-1: {baseline_results['pets']['top1_accuracy']:.4f}")
    print(f"  - Post-Finetune Top-1: {post_results['pets']['top1_accuracy']:.4f}")
    print(f"  - Differenza: {pet_top1_diff:+.4f} (Obiettivo: > -0.02)")

    print(f"\n🎯 Bias di Associazione (Pexels - Immagini Facili):")
    print(f"  - Baseline Bias Assoluto: {baseline_results['pexels_bias']['overall_abs_bias']:.4f}")
    print(f"  - Post-Finetune Bias Assoluto: {post_results['pexels_bias']['overall_abs_bias']:.4f}")
    print(f"  - Riduzione del Bias: {bias_perc_reduction:.2f}% (Obiettivo: Alto)")

    print(f"\n🎯 Bias di Associazione (Challenge Set - Immagini Ambigue):") # <-- BLOCCO AGGIUNTO
    print(f"  - Baseline Bias Assoluto: {baseline_results['challenge_bias']['overall_abs_bias']:.4f}")
    print(f"  - Post-Finetune Bias Assoluto: {post_results['challenge_bias']['overall_abs_bias']:.4f}")
    print(f"  - Riduzione del Bias: {challenge_bias_perc_reduction:.2f}% (Obiettivo: Alto)")

    print(f"\n🎯 Generalizzazione (Pexels Hold-out):")
    print(f"  - Bias Post (Standard): {post_results['pexels_bias']['overall_abs_bias']:.4f}")
    print(f"  - Bias Post (Hold-out): {post_results['pexels_bias_holdout']['overall_abs_bias']:.4f}")
    if post_results['pexels_bias_holdout']['overall_abs_bias'] < baseline_results['pexels_bias']['overall_abs_bias']:
        print("  - ✔️ Successo: Il debiasing si è generalizzato a nuovi template.")
    else:
        print("  - ⚠️ Attenzione: Overfitting sui template di training?")

    print(f"\n🎯 Bias di Rappresentazione (FairFace):")
    print(f"  - Baseline Acc. Genere: {baseline_results['fairface_bias']['overall_gender_acc']:.4f}")
    print(f"  - Post-Finetune Acc. Genere: {post_results['fairface_bias']['overall_gender_acc']:.4f}")
    print(f"  - Differenza: {ff_gender_diff:+.4f} (Obiettivo: ~0.0)")

    print("\nVisualizzazioni PCA:")
    print("  - Confronta i due plot 'PCA Spazio Testuale' (Step 3 e 6).")
    print("  - Ci aspettiamo di vedere 'engineer', 'nurse', 'doctor' muoversi verso")
    print("    una posizione più centrale rispetto agli assi di genere.")

except Exception as e:
    print(f"Errore nella generazione delle conclusioni quantitative: {e}")

print("\n" + "=" * 50)
print("SCRIPT COMPLETO ESEGUITO")
print("=" * 50)